# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
text.split('\n')[1]

'Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.'

In [4]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    total_counts = Counter(text)
   
    words = sorted(total_counts, key=total_counts.get)
    vocab_to_int = {}
    for num,word in enumerate(words):
        vocab_to_int[word] = num
    int_to_vocab = {num: word for word, num in vocab_to_int.items()}
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


#### Attributions 

https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value


https://stackoverflow.com/questions/483666/python-reverse-invert-a-mapping

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token = {}
    token['.'] = "||Period||"
    token[','] = "||Comma||"
    token['"'] = "||QuotationMark||"
    token[';'] = "||Semicolon||"
    token['!'] = "||ExclamationMark||"
    token['?'] = "||QuestionMark||"
    token['('] = "||LeftParentheses||"
    token[')'] = "||RightParentheses||"
    token['--'] = "||Dash||"
    token['\n'] = "||Return||"
       
    # For future use 
    
    #token[''] = " "
    #token[''] = " "
    return token

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [9]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    Inputs = tf.placeholder(tf.int32, [None, None], name='input')
    Targets = tf.placeholder(tf.int32, [None, None], name='labels')
    LearningRate = tf.placeholder(tf.float32, name='learning_rate')
    return Inputs, Targets, LearningRate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [11]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=0.80)
    Cell = tf.contrib.rnn.MultiRNNCell([drop])
    initial_state = Cell.zero_state(batch_size, tf.float32)
    InitialState = tf.identity(initial_state, name='initial_state')
    return Cell, InitialState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


#### Attributions

https://www.oreilly.com/learning/perform-sentiment-analysis-with-lstms-using-tensorflow

### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [15]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embeds = tf.contrib.layers.embed_sequence(input_data, vocab_size, embed_dim)

    return embeds


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


#### Articles

[Vector Representations of Words ](https://www.tensorflow.org/tutorials/word2vec)



### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [16]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    
    """
    # TODO: Implement Function
    
    Outputs, state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    FinalState = tf.identity(state, name="final_state")

    return Outputs, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [17]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    
    Embeds = get_embed(input_data, vocab_size, embed_dim)
    Outputs, FinalState = build_rnn(cell, Embeds)

    Logits = tf.contrib.layers.fully_connected(Outputs,vocab_size,activation_fn=None, normalizer_fn=None, 
                                               trainable=True, scope=None)

    return Logits, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


#### Attributions 

Most articles were provided by the 1st Reviewer(Thank you)

https://www.quora.com/Is-ReLU-a-linear-piece-wise-linear-or-non-linear-activation-function

https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [22]:
from itertools import cycle

def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    n_batches = int(len(int_text)/(batch_size*seq_length))
    
    # only full batches
    X = np.array(int_text[:n_batches*batch_size*seq_length])
    Y = np.array(int_text[1:n_batches*batch_size*seq_length])
    Y = np.append(Y, X[0])
    
    X_batches = np.split(X.reshape(batch_size, -1), n_batches, 1)
    Y_batches = np.split(Y.reshape(batch_size, -1), n_batches, 1)
    
    
    return np.array((list(zip(X_batches, Y_batches))))
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [26]:
# Number of Epochs
num_epochs = 200
# Batch Size
batch_size =64
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 200
# Sequence Length
seq_length = 14
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 16
"""
Epoch  99 Batch   29/33   train_loss = 2.588
epoch =100, batch_size = 128,  rnn_size = 512, embed_size = 100, seq_length = 16, learning rate = 0.001 

Epoch 199 Batch    0/8   train_loss = 2.342
epoch =200, batch_size = 256,  rnn_size = 1024, embed_size = 500, seq_length = 32, learning rate = 0.001




"""
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [27]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/77   train_loss = 8.821
Epoch   0 Batch   16/77   train_loss = 6.971
Epoch   0 Batch   32/77   train_loss = 6.307
Epoch   0 Batch   48/77   train_loss = 6.224
Epoch   0 Batch   64/77   train_loss = 6.110
Epoch   1 Batch    3/77   train_loss = 6.058
Epoch   1 Batch   19/77   train_loss = 6.031
Epoch   1 Batch   35/77   train_loss = 5.991
Epoch   1 Batch   51/77   train_loss = 5.912
Epoch   1 Batch   67/77   train_loss = 5.899
Epoch   2 Batch    6/77   train_loss = 5.726
Epoch   2 Batch   22/77   train_loss = 5.627
Epoch   2 Batch   38/77   train_loss = 5.527
Epoch   2 Batch   54/77   train_loss = 5.538
Epoch   2 Batch   70/77   train_loss = 5.271
Epoch   3 Batch    9/77   train_loss = 5.446
Epoch   3 Batch   25/77   train_loss = 5.391
Epoch   3 Batch   41/77   train_loss = 5.311
Epoch   3 Batch   57/77   train_loss = 5.187
Epoch   3 Batch   73/77   train_loss = 5.138
Epoch   4 Batch   12/77   train_loss = 5.264
Epoch   4 Batch   28/77   train_loss = 5.251
Epoch   4 

Epoch  38 Batch    2/77   train_loss = 2.555
Epoch  38 Batch   18/77   train_loss = 2.376
Epoch  38 Batch   34/77   train_loss = 2.459
Epoch  38 Batch   50/77   train_loss = 2.609
Epoch  38 Batch   66/77   train_loss = 2.544
Epoch  39 Batch    5/77   train_loss = 2.618
Epoch  39 Batch   21/77   train_loss = 2.474
Epoch  39 Batch   37/77   train_loss = 2.514
Epoch  39 Batch   53/77   train_loss = 2.492
Epoch  39 Batch   69/77   train_loss = 2.398
Epoch  40 Batch    8/77   train_loss = 2.496
Epoch  40 Batch   24/77   train_loss = 2.490
Epoch  40 Batch   40/77   train_loss = 2.478
Epoch  40 Batch   56/77   train_loss = 2.427
Epoch  40 Batch   72/77   train_loss = 2.545
Epoch  41 Batch   11/77   train_loss = 2.380
Epoch  41 Batch   27/77   train_loss = 2.451
Epoch  41 Batch   43/77   train_loss = 2.378
Epoch  41 Batch   59/77   train_loss = 2.504
Epoch  41 Batch   75/77   train_loss = 2.471
Epoch  42 Batch   14/77   train_loss = 2.381
Epoch  42 Batch   30/77   train_loss = 2.395
Epoch  42 

Epoch  76 Batch    4/77   train_loss = 1.510
Epoch  76 Batch   20/77   train_loss = 1.393
Epoch  76 Batch   36/77   train_loss = 1.303
Epoch  76 Batch   52/77   train_loss = 1.409
Epoch  76 Batch   68/77   train_loss = 1.429
Epoch  77 Batch    7/77   train_loss = 1.300
Epoch  77 Batch   23/77   train_loss = 1.455
Epoch  77 Batch   39/77   train_loss = 1.389
Epoch  77 Batch   55/77   train_loss = 1.490
Epoch  77 Batch   71/77   train_loss = 1.589
Epoch  78 Batch   10/77   train_loss = 1.509
Epoch  78 Batch   26/77   train_loss = 1.581
Epoch  78 Batch   42/77   train_loss = 1.389
Epoch  78 Batch   58/77   train_loss = 1.455
Epoch  78 Batch   74/77   train_loss = 1.468
Epoch  79 Batch   13/77   train_loss = 1.457
Epoch  79 Batch   29/77   train_loss = 1.469
Epoch  79 Batch   45/77   train_loss = 1.402
Epoch  79 Batch   61/77   train_loss = 1.588
Epoch  80 Batch    0/77   train_loss = 1.450
Epoch  80 Batch   16/77   train_loss = 1.412
Epoch  80 Batch   32/77   train_loss = 1.394
Epoch  80 

Epoch 114 Batch    6/77   train_loss = 0.924
Epoch 114 Batch   22/77   train_loss = 0.841
Epoch 114 Batch   38/77   train_loss = 0.847
Epoch 114 Batch   54/77   train_loss = 0.859
Epoch 114 Batch   70/77   train_loss = 0.951
Epoch 115 Batch    9/77   train_loss = 0.842
Epoch 115 Batch   25/77   train_loss = 0.890
Epoch 115 Batch   41/77   train_loss = 0.949
Epoch 115 Batch   57/77   train_loss = 0.878
Epoch 115 Batch   73/77   train_loss = 0.911
Epoch 116 Batch   12/77   train_loss = 0.814
Epoch 116 Batch   28/77   train_loss = 0.804
Epoch 116 Batch   44/77   train_loss = 0.799
Epoch 116 Batch   60/77   train_loss = 0.846
Epoch 116 Batch   76/77   train_loss = 0.821
Epoch 117 Batch   15/77   train_loss = 0.867
Epoch 117 Batch   31/77   train_loss = 0.747
Epoch 117 Batch   47/77   train_loss = 0.731
Epoch 117 Batch   63/77   train_loss = 0.839
Epoch 118 Batch    2/77   train_loss = 0.788
Epoch 118 Batch   18/77   train_loss = 0.831
Epoch 118 Batch   34/77   train_loss = 0.716
Epoch 118 

Epoch 152 Batch    8/77   train_loss = 0.545
Epoch 152 Batch   24/77   train_loss = 0.667
Epoch 152 Batch   40/77   train_loss = 0.625
Epoch 152 Batch   56/77   train_loss = 0.543
Epoch 152 Batch   72/77   train_loss = 0.617
Epoch 153 Batch   11/77   train_loss = 0.532
Epoch 153 Batch   27/77   train_loss = 0.592
Epoch 153 Batch   43/77   train_loss = 0.555
Epoch 153 Batch   59/77   train_loss = 0.541
Epoch 153 Batch   75/77   train_loss = 0.557
Epoch 154 Batch   14/77   train_loss = 0.571
Epoch 154 Batch   30/77   train_loss = 0.556
Epoch 154 Batch   46/77   train_loss = 0.590
Epoch 154 Batch   62/77   train_loss = 0.576
Epoch 155 Batch    1/77   train_loss = 0.625
Epoch 155 Batch   17/77   train_loss = 0.559
Epoch 155 Batch   33/77   train_loss = 0.511
Epoch 155 Batch   49/77   train_loss = 0.593
Epoch 155 Batch   65/77   train_loss = 0.591
Epoch 156 Batch    4/77   train_loss = 0.592
Epoch 156 Batch   20/77   train_loss = 0.494
Epoch 156 Batch   36/77   train_loss = 0.463
Epoch 156 

Epoch 190 Batch   10/77   train_loss = 0.464
Epoch 190 Batch   26/77   train_loss = 0.522
Epoch 190 Batch   42/77   train_loss = 0.411
Epoch 190 Batch   58/77   train_loss = 0.401
Epoch 190 Batch   74/77   train_loss = 0.416
Epoch 191 Batch   13/77   train_loss = 0.428
Epoch 191 Batch   29/77   train_loss = 0.466
Epoch 191 Batch   45/77   train_loss = 0.403
Epoch 191 Batch   61/77   train_loss = 0.473
Epoch 192 Batch    0/77   train_loss = 0.485
Epoch 192 Batch   16/77   train_loss = 0.383
Epoch 192 Batch   32/77   train_loss = 0.429
Epoch 192 Batch   48/77   train_loss = 0.438
Epoch 192 Batch   64/77   train_loss = 0.398
Epoch 193 Batch    3/77   train_loss = 0.416
Epoch 193 Batch   19/77   train_loss = 0.381
Epoch 193 Batch   35/77   train_loss = 0.375
Epoch 193 Batch   51/77   train_loss = 0.451
Epoch 193 Batch   67/77   train_loss = 0.413
Epoch 194 Batch    6/77   train_loss = 0.469
Epoch 194 Batch   22/77   train_loss = 0.384
Epoch 194 Batch   38/77   train_loss = 0.422
Epoch 194 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [29]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [30]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [31]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name('input:0')
    InitialStateTensor = loaded_graph.get_tensor_by_name('initial_state:0')
    FinalStateTensor = loaded_graph.get_tensor_by_name('final_state:0')
    ProbsTensor = loaded_graph.get_tensor_by_name('probs:0')
    return (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [49]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    return (int_to_vocab[(np.argmax(probabilities))])


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [50]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak: well, let me check the bar!
moe_szyslak:(absentminded, to be the bartender, moe, eh, the dank. and like on, but i'm gonna go home to the past / get your kill a man clean...(dramatic) the mouse are...
moe_szyslak:(cutting him off) yeah, yeah, yeah, yeah!(suspicious) all because i forgot so much-- but mean to the best guy from a man for me with me.
lenny_leonard: you know, homer, you think we're just just have to have.


homer_simpson: hey! quit harvesting me out. and charge it!
moe_szyslak:(looking at homer) you're right, moe. you're not a picky man.(burp)
homer_simpson:(running out, doppler) the queen's private candidate, moe.
homer_simpson: oh, i'm so proud of moe's.


moe_szyslak:(excited) yeah, yeah, yeah. you gotta get back.


homer_simpson:(tipsy) man, i


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.